### Use older version of OpenCV-python.

In [ ]:
pip install opencv-contrib-python==4.6.0.66

### Stable version, jumping images


In [ ]:
import cv2
import cv2.aruco as aruco
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/ArUco_marker.mp4')
cap1 = cv2.VideoCapture('/content/bean.mp4')

if not all([cap.isOpened(), cap1.isOpened()]):
    print("Error opening video files!")
    exit()

#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX

arucoDict    = aruco.Dictionary_get(aruco.DICT_7X7_50)

# 必須描述ChArUco board的尺寸規格
gridX        = 2 # 水平方向5格
gridY        = 3 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
# ArUco marker為2cmX2cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
refinedStrategy = True
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
frameId        = 0
collectCorners = []
collectIds     = []
collectFrames  = []

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4',fourcc,20.0,(frameWidth,frameHeight))


while True:
    ret, frame = cap.read()
    ret1, frame1 = cap1.read()

    if not all([ret,ret1]):
        break

    print(frame1.shape)
    print("--")

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    #print(corners)

    h1,w1,dim = frame1.shape
    pst_src = np.array(
        [
            [w1 - 1, h1 -1],
            [0, h1 -1],
            [0,0],
            [w1 - 1,0],
        ],dtype = float
    )
    print("pst_src",pst_src)
    points_2d = np.vstack(corners[0]).astype(float)

    print("corners[0] =",corners[0])
    print("points_2d =", points_2d)
    h1, status = cv2.findHomography(pst_src,points_2d)

    print("h1 =",h1)
    print(frame1.shape)

    print("%%")

    frame1 = cv2.warpPerspective(frame1,h1,(frame.shape[1],frame.shape[0]))

    print(frame1.shape)
    print("---")
    cv2.fillConvexPoly(frame,corners[0].astype(int),0,16)

    print(frame.shape)
    print("---")
    print(frame1.shape)
    frame = cv2.add(frame,frame1)

    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)

    if frameId % 100 == 50 and ids is not None and len(ids)==6: # 6 ArUco markers
        collectCorners.append(corners)
        collectIds.append(ids.ravel())
        collectFrames.append(frame)

    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)

    #cv2_imshow(frame)
    out.write(frame)

    frameId += 1

cv2.destroyAllWindows()
cap.release()
cap1.release()
#print(collectCorners)

### Final version for part 1 - ARUco marker.

In [ ]:
def secretRecipe(corners, id, frame, frameAug, drawId = True):
  print(corners)
  btml = corners[0][0][0], corners[0][0][1]
  btmr = corners[0][1][0], corners[0][1][1]
  topl = corners[0][2][0], corners[0][2][1]
  topr = corners[0][3][0], corners[0][3][1]
  h,w, dim = frameAug.shape
  dest = np.array([btml,btmr,topl,topr]).astype(float)
  src = np.array([[w - 1, h -1],
                  [0, h -1],
                  [0,0],
                  [w - 1,0]],dtype = float)

  h, status = cv2.findHomography(src,dest)
  frame1 = cv2.warpPerspective(frameAug,h,(frame.shape[1],frame.shape[0]))
  cv2.fillConvexPoly(frame,dest.astype(int),(0,0,0))
  return cv2.add(frame,frame1)

In [ ]:
import cv2
import cv2.aruco as aruco
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/ArUco_marker.mp4')
cap1 = cv2.VideoCapture('/content/bean.mp4')
cap2 = cv2.VideoCapture('/content/rU_livin_day2day.mp4')
cap3 = cv2.VideoCapture('/content/BlendW.mp4')
cap4 = cv2.VideoCapture('/content/tacica-Leo.mp4')
cap5 = cv2.VideoCapture('/content/pvsSisyphus.mp4')
cap6 = cv2.VideoCapture('/content/JpHorse_getDown.mp4')

if not all([cap.isOpened(), cap1.isOpened(), cap2.isOpened(),cap3.isOpened(), cap4.isOpened(), cap5.isOpened()]):
    print("Error opening video files!")
    exit()

#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

arucoParams  = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
arucoDict    = aruco.Dictionary_get(aruco.DICT_7X7_50)

# 必須描述ChArUco board的尺寸規格
gridX        = 2 # 水平方向5格
gridY        = 3 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
# ArUco marker為2cmX2cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)
print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
refinedStrategy = True
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
frameId        = 0

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc,30.0,(frameWidth,frameHeight))

while True:
    ret, frame = cap.read()
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()
    ret4, frame4 = cap4.read()
    ret5, frame5 = cap5.read()
    ret6, frame6 = cap6.read()

    if not all([ret,ret1,ret2,ret3,ret4,ret5,ret6]):
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)

    if ids is not None and len(ids)==6: # 6 ArUco markers
      for corners, ids in zip(corners, ids):
        if ids == 1:
          frame = secretRecipe(corners, ids, frame, frame1)
        if ids == 2:
          frame = secretRecipe(corners, ids, frame, frame2)
        if ids == 3:
          frame = secretRecipe(corners, ids, frame, frame3)
        if ids == 4:
          frame = secretRecipe(corners, ids, frame, frame4)
        if ids == 5:
          frame = secretRecipe(corners, ids, frame, frame5)
        if ids == 6:
          frame = secretRecipe(corners, ids, frame, frame6)

    #cv2_imshow(frame)
    out.write(frame)

    frameId += 1

cv2.destroyAllWindows()
cap.release()
cap1.release()
#print(collectCorners)

### Part 2 - CharARUco detection.

In [ ]:
import cv2
import cv2.aruco as aruco

cap = cv2.VideoCapture('/content/ChArUco_board.mp4')
print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
arucoDict    = aruco.Dictionary_get(aruco.DICT_6X6_250)
gridX        = 5
gridY        = 7
squareSize   = 4 #cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.avi',fourcc,30.0,(frameWidth,frameHeight))
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if ids is not None and len(ids)>0:
        aruco.drawDetectedMarkers(frame, corners, ids)
        ret, rvect, tvect = aruco.estimatePoseBoard(corners, ids, charucoBoard, charuco_cameraMatrix, charuco_distCoeffs, None, None)
#         if ret:
#             aruco.drawAxis(frame, charuco_cameraMatrix, charuco_distCoeffs, rvect, tvect, squareSize)

    #cv2_imshow(frame)
    out.write(frame)

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0


In [ ]:
import cv2
import cv2.aruco as aruco

cap = cv2.VideoCapture('/content/ChArUco_board.mp4')
cap1 = cv2.VideoCapture('/content/kcGun.mp4')
cap2 = cv2.VideoCapture('/content/tacica-Leo.mp4')
cap3 = cv2.VideoCapture('/content/whatIveDone.mp4')
cap4 = cv2.VideoCapture('/content/BlendW.mp4')

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
arucoDict    = aruco.Dictionary_get(aruco.DICT_6X6_250)
gridX        = 5
gridY        = 7
squareSize   = 4 #cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.avi',fourcc,30.0,(frameWidth,frameHeight))
cameraMatrixInit = np.array([[ frameWidth/2.,    0., frameWidth/2.],[    0., frameWidth/2., frameHeight/2.],[    0.,    0.,           1.]])
distCoeffsInit   = np.zeros((5,1))

while True:
    ret, frame = cap.read()
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()
    ret4, frame4 = cap4.read()
    if not all([ret, ret1, ret2, ret3, ret4]):
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if ids is not None and len(ids)>0:
        aruco.drawDetectedMarkers(frame, corners, ids)
        ret, rvect, tvect = aruco.estimatePoseBoard(corners, ids, charucoBoard, charuco_cameraMatrix, charuco_distCoeffs, None, None)
        if ret:
            cv2.drawFrameAxes(frame, cameraMatrixInit, distCoeffsInit, rvect, tvect, squareSize)

        for corners, ids in zip(corners, ids):
          if ids % 4 == 0:
            frame = secretRecipe(corners, ids, frame, frame1)
          if ids % 4 == 1:
            frame = secretRecipe(corners, ids, frame, frame2)
          if ids % 4 == 2:
            frame = secretRecipe(corners, ids, frame, frame3)
          if ids % 4 == 3:
            frame = secretRecipe(corners, ids, frame, frame4)



    #cv2_imshow(frame)
    out.write(frame)

cv2.destroyAllWindows()
cap.release()
cap1.release()
cap2.release()
cap3.release()
cap4.release()